In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import fisher_exact
from scipy.stats import barnard_exact
from scipy.stats import binom
from scipy.stats import norm

In [ ]:
# A function that performs the two-sided proportion test using the test statistic t that we defined in the blog post.
# It takes as input the group sizes n_a and n_b and the number of successess x_a and x_b from each group.
# It returns a p-value that is computed using the approximation that t follows a standard normal distribution.

def two_sided_prop_test(x_a, x_b, n_a, n_b):
    r_a = x_a / n_a
    r_b = x_b / n_b 
    r = (x_a + x_b) / (n_a + n_b)
    
    if ((x_a == 0 and x_b == 0) or (x_a == n_a and x_b == n_b) or (x_a == n_b and x_b == n_a)):
        t = 0
    else:
        t = (r_a - r_b) / np.sqrt(r * (1 - r) * (1 / n_a + 1 / n_b))
            
    if t <= 0:
        p_val = 2 * norm.cdf(t)
    else:
        p_val = 2 * (1 - norm.cdf(t))

    return p_val

In [ ]:
# Compute the type 1 error rate of the approximate test two_sided_prop_test when the samples are small.
# We assume the null hypothesis that the probability of a success is the same for both groups.
# We set the sizes of both groups to be the same for simplicity.

# For each sample size and each value of the common success probability, we run n_repeats experiments in which we
# draw two samples from the same distribution and then test the null hypothesis that the distributions are the same using the 
# approximate test. We then record the error rate (false positive rate) which is equal to the number of false positives 
# (i.e., the number of times the test result was significant) divided by the number of repetitions of the experiment.

sample_sizes = [5, 10, 15, 20, 25, 30] 
success_probabilities = [0.1, 0.2, 0.3, 0.4, 0.5]
alpha = 0.05

error_rates = [[], [], [], [], []]

n_repeats = 10000

for n in sample_sizes:
    
    n_a = n
    n_b = n
    
    j = 0
    
    for p in success_probabilities:
    
        # Repeat the following n_repeats times.
        significant = 0
        
        for i in range(n_repeats):
            
            # Draw two samples from the same binomial distribution.
            x_a = binom.rvs(n_a, p, size = 1)[0]
            x_b = binom.rvs(n_b, p, size = 1)[0]
            
            # Test if the two distributions are different according to the approximate test.
            p_val = two_sided_prop_test(x_a, x_b, n_a, n_b)
                
            if p_val <= alpha:
                significant += 1  
        
        # Here, significant / n_repeats is the fraction of times that we got a false positive result.
        error_rates[j].append(significant / n_repeats)
        
        j += 1
        
print(error_rates)

In [ ]:
# Generate a figure that shows the results of the experiment above. For each sample size and each common success probability,
# show the actual type 1 error rate that we found in the experiment.

for item in error_rates:
    plt.plot(sample_sizes, item, marker = 'o', linestyle = '-')

plt.axis([0, 35, 0, 0.07])
plt.xlabel('n (size of each group)')    
plt.ylabel('Actual Type 1 Error Rate')
plt.legend([r'$p = 0.1$', r'$p = 0.2$', r'$p = 0.3$', r'$p = 0.4$', r'$p = 0.5$'])
plt.text(30.5, 0.067, r'$\alpha = 0.05$')
plt.show()

In [ ]:
# Power calculation for Fisher's exact test and Barnard's exact test.
# The power of a test against a specific alternative hypothesis is the probability that the test result is
# significant when the alternative hypothesis is true.
# We again set the sizes of both groups to be the same for simplicity.

sample_sizes = [5, 10, 15, 20, 25, 30] 
p_a = 0.1 # 0.5
p_b = 0.2 # 0.6
alpha = 0.05

n_repeats = 1000

fisher_power = []
barnard_power = []

for n in sample_sizes:
    n_a = n
    n_b = n
    
    # Repeat the following n_repeats times.
    fisher_success = 0
    barnard_success = 0
    
    for i in range(n_repeats):
        # Draw two samples from the two different binomial distributions.
        x_a = binom.rvs(n, p_a, size = 1)[0]
        x_b = binom.rvs(n, p_b, size = 1)[0]
        
        # Prepare the contingency table M that is fed to the fisher_exact and barnard_exact functions.
        M = [[x_a, x_b], [n_a - x_a, n_b - x_b]]
        
        # Test if they are different using Fisher's test.
        res_fisher = fisher_exact(M, alternative = 'two-sided')
        
        if res_fisher[1] <= alpha:
            fisher_success += 1
        
        # Test if they are different using Barnard's test.
        res_barnard = barnard_exact(M, alternative = 'two-sided')
        
        if res_barnard.pvalue <= alpha:
            barnard_success += 1
            
    fisher_power.append(fisher_success / n_repeats)
    barnard_power.append(barnard_success / n_repeats)
        
    
print(fisher_power)
print(barnard_power)

In [ ]:
## Make a nice plot of the power results with the following parameters:
## p_A = 0.5, p_B = 0.6, alpha = 0.05
#
#plt.plot(sample_sizes, fisher_power, sample_sizes, barnard_power, marker = 'o', linestyle = '-')
#
#plt.axis([0, 35, 0, 0.15])
#plt.text(1, 0.14, r'$p_A = 0.5$, $p_B = 0.6$, $\alpha = 0.05$')
#plt.legend(['Fisher', 'Barnard'])
#plt.xlabel('n (size of each group)')
#plt.ylabel('Power')
#
#plt.show()

In [ ]:
# Make a nice plot of the power results with the following parameters:
# p_A = 0.1, p_B = 0.2, alpha = 0.05

plt.plot(sample_sizes, fisher_power, sample_sizes, barnard_power, marker = 'o', linestyle = '-')

plt.axis([0, 35, 0, 0.20])
plt.text(1, 0.19, r'$p_A = 0.1$, $p_B = 0.2$, $\alpha = 0.05$')
plt.legend(['Fisher', 'Barnard'])
plt.xlabel('n (size of each group)')
plt.ylabel('Power')

plt.show()